In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:99%;}
div.CodeMirror {font-family:Consolas; font-size:20pt;}
div.output {font-size:18pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:19pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
span.toc-item-num{display:none;}
div.text_cell_render ul li{font-size:16pt;padding:5px;}
div.CodeMirror-lines > div {padding-left:10px;}
table.dataframe{font-size:19px;}
</style>
"""))

# 인코더 LSTM과 디코더 LSTM으로 스마트 번역기 만들기

In [33]:
# 1. 패키지 수입
import numpy as np
import pandas as pd
from time import time

from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

# 하이퍼 파라미터 (정확도나 속도에 영향을 미치는 값)
MY_HIDDEN = 128
MY_EPOCH  = 500

In [8]:
# 2. 번역 데이터 불러오기
raw = pd.read_csv('trans_dataset/translate.csv', header=None)
eng_kor = raw.values.tolist() # 데이터 프레임을 list로 변환
print(len(eng_kor))
print(eng_kor[:3])

110
[['cold', '감기'], ['come', '오다'], ['cook', '요리']]


In [22]:
# 3. 영어알파벳과 한글알파벳 리스트 만들기

e_alpha = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz']
# {c:i for i, c in enumerate(e_alpha)}
k_alpha = pd.read_csv('trans_dataset/korean.csv', header=None)[0].tolist()
alpha = e_alpha + k_alpha
print('영어와 한글 알파벳 : ', alpha)
alpha_total_size = len(alpha)
print('전체 알파벳 갯수(원핫인코딩할 size) :', alpha_total_size)
print('영어 알파벳 갯수 :', len(e_alpha))
print('한글 글자수 갯수 :', len(k_alpha))

영어와 한글 알파벳 :  ['S', 'E', 'P', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '가', '각', '간', '감', '개', '거', '것', '게', '계', '고', '관', '광', '구', '굴', '규', '그', '금', '기', '깊', '나', '날', '남', '내', '넓', '녀', '노', '놀', '농', '높', '뉴', '늦', '다', '단', '도', '동', '들', '람', '랑', '래', '램', '류', '름', '릎', '리', '많', '망', '매', '머', '먼', '멍', '메', '명', '모', '목', '무', '물', '미', '바', '반', '방', '번', '복', '부', '분', '붕', '비', '뿌', '사', '상', '색', '생', '서', '선', '소', '손', '수', '쉽', '스', '시', '식', '실', '싸', '아', '약', '얇', '어', '언', '얼', '여', '연', '오', '옥', '왼', '요', '용', '우', '운', '움', '위', '유', '은', '을', '음', '의', '이', '익', '인', '읽', '입', '자', '작', '장', '적', '제', '좋', '주', '지', '짜', '쪽', '찾', '책', '출', '칙', '크', '키', '탈', '택', '통', '파', '팔', '편', '피', '핑', '한', '합', '해', '행', '험', '회', '획', '휴', '흐']
전체 알파벳 갯수(원핫인코딩할 size) : 171
영어 알파벳 갯수 : 29
한글 글자수 갯수 : 142


In [24]:
# 문자당 num를 갖는 dict 만들기
char_to_num = {c:i for i, c in enumerate(alpha)}
print(char_to_num)

{'S': 0, 'E': 1, 'P': 2, 'a': 3, 'b': 4, 'c': 5, 'd': 6, 'e': 7, 'f': 8, 'g': 9, 'h': 10, 'i': 11, 'j': 12, 'k': 13, 'l': 14, 'm': 15, 'n': 16, 'o': 17, 'p': 18, 'q': 19, 'r': 20, 's': 21, 't': 22, 'u': 23, 'v': 24, 'w': 25, 'x': 26, 'y': 27, 'z': 28, '가': 29, '각': 30, '간': 31, '감': 32, '개': 33, '거': 34, '것': 35, '게': 36, '계': 37, '고': 38, '관': 39, '광': 40, '구': 41, '굴': 42, '규': 43, '그': 44, '금': 45, '기': 46, '깊': 47, '나': 48, '날': 49, '남': 50, '내': 51, '넓': 52, '녀': 53, '노': 54, '놀': 55, '농': 56, '높': 57, '뉴': 58, '늦': 59, '다': 60, '단': 61, '도': 62, '동': 63, '들': 64, '람': 65, '랑': 66, '래': 67, '램': 68, '류': 69, '름': 70, '릎': 71, '리': 72, '많': 73, '망': 74, '매': 75, '머': 76, '먼': 77, '멍': 78, '메': 79, '명': 80, '모': 81, '목': 82, '무': 83, '물': 84, '미': 85, '바': 86, '반': 87, '방': 88, '번': 89, '복': 90, '부': 91, '분': 92, '붕': 93, '비': 94, '뿌': 95, '사': 96, '상': 97, '색': 98, '생': 99, '서': 100, '선': 101, '소': 102, '손': 103, '수': 104, '쉽': 105, '스': 106, '시': 107, '식': 108, '실': 109, '싸': 110,

In [32]:
eng_kor[0]
print(char_to_num['c'], char_to_num['o'], char_to_num['l'], char_to_num['d'])
print([char_to_num[c] for c in eng_kor[0][0]])
print(char_to_num['감'], char_to_num['기'])
print([char_to_num[c] for c in eng_kor[0][1]])

5 17 14 6
[5, 17, 14, 6]
32 46
[32, 46]


In [38]:
# 원핫인코딩 방법1 : 이 코드에서는 적용 힘듦. display(pd.get_dummies([5, 3, 6]))
# 원핫인코딩 방법2
to_categorical([5, 3, 6], num_classes=10) # 0~9인덱스 원핫인코딩

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]], dtype=float32)

In [41]:
# 원핫인코딩 방법3 : 단위행렬을 이용한 방법
np.eye(10)[[5,3,6]]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]])

In [42]:
# 5. 인코더 입력(영어알파벳을 숫자로 바꾼 값을 원핫인코딩), 
#    디코더 입력("S한글"을 숫자로 바꾼 값을 원핫인코딩)
#    디코더 출력("한글E"을 숫자로 바꾼 값)

[['cold', '감기'],
 ['come', '오다'],
 ['cook', '요리'],
 ['copy', '복사'],
 ['cost', '비용'],
 ['date', '날짜'],
 ['deal', '거래'],
 ['deep', '깊은'],
 ['desk', '책상'],
 ['down', '아래'],
 ['dust', '먼지'],
 ['duty', '의무'],
 ['each', '각각'],
 ['east', '동쪽'],
 ['easy', '쉽다'],
 ['exit', '탈출'],
 ['face', '얼굴'],
 ['fact', '사실'],
 ['fall', '가을'],
 ['farm', '농장'],
 ['feet', '다리'],
 ['find', '찾다'],
 ['fine', '좋다'],
 ['fish', '생선'],
 ['flow', '흐름'],
 ['fund', '기금'],
 ['gain', '수익'],
 ['game', '놀이'],
 ['gift', '선물'],
 ['girl', '소녀'],
 ['give', '주다'],
 ['goal', '목적'],
 ['gray', '회색'],
 ['hair', '머리'],
 ['harm', '피해'],
 ['hell', '지옥'],
 ['help', '도움'],
 ['high', '높은'],
 ['hole', '구멍'],
 ['hope', '소망'],
 ['hour', '시각'],
 ['join', '합류'],
 ['kiss', '키스'],
 ['knee', '무릎'],
 ['lady', '부인'],
 ['late', '늦은'],
 ['left', '왼쪽'],
 ['life', '생명'],
 ['loss', '손해'],
 ['love', '사랑'],
 ['luck', '행운'],
 ['mail', '우편'],
 ['male', '남자'],
 ['many', '많은'],
 ['meal', '식사'],
 ['meat', '고기'],
 ['menu', '메뉴'],
 ['milk', '우유'],
 ['name', '이름'